In [2]:
import os
import sys
import pickle
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sn
from pprint import pprint
import statsmodels.api as sms
from pyd3 import interpolate, color
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from SensorBlockchainNetwork.analyze import *
# from SensorBlockchainNetwork.SensorBlockchainNetwork import *

plt = matplotlib.pyplot
plt.style.use('seaborn-muted')
color_interpolator = interpolate.rgb('blue', 'red')

In [ ]:
files = [file for file in os.listdir('./data/final/') if 'pickles' not in file and 'fixed' not in file]
model_dataframes = {}

In [ ]:
for file in files:

    model_dataframes[file.replace('.pkl', '')] = get_model_vars_df_from_pickle('./data/final/' + file)

## Investigating effect of network size on dependent variables

### Effects on mining dynamics

In [ ]:
num_sensors_results = {
    'mining': {}, 
    'info_currency': {},
    'gwei_spent': {}
}

In [ ]:
# num_sensors_model_df = model_dataframes['batch_run_1_num_sensors']
num_sensors_model_df = get_model_vars_df_from_pickle('./data/final/batch_run_1_num_sensors.pkl')

In [ ]:
num_sensors_model_df[['mining_mean']].describe()

In [ ]:
num_sensors_model_df[['num_sensors', 'mining_mean']].corr()

In [ ]:
plt.figure(figsize=(10,4))
plt.scatter(num_sensors_model_df['num_sensors'], num_sensors_model_df['mining_mean'])


In [ ]:
num_sensors_mining_OLS = SimpleOLSRegression(num_sensors_model_df, 'num_sensors', "Network size (number of sensors)", 'mining_mean', 'Mean transaction mining time (blocks)')

In [ ]:
num_sensors_mining_OLS.scatter_OLS()#'./outputs/figures/num-sensors/mining-mean/scatter-OLS-full.svg')


In [ ]:
num_sensors_mining_OLS.plot_residuals_fits()#'./outputs/figures/num-sensors/mining-mean/residuals-fits-OLS-full.svg')


In [ ]:
plt.hist(num_sensors_model_df['mining_mean'])
plt.xlabel('Mean mining time per block')
plt.ylabel('Count, model runs')
_ = plt.title("Mean mining time per block for model runs (n=30)")
# plt.savefig('./outputs/figures/num-sensors/mining-mean/hist.svg')

### Interpretation

Are LINE conditions met? 

Linear: It does not appear so, though roughly, yes.
Independent 

In [ ]:
num_sensors_results['mining']['pearson'] = num_sensors_model_df[['num_sensors', 'mining_mean']].corr()

num_sensors_results['mining']['X_values'] = sms.add_constant(num_sensors_model_df['num_sensors'])
num_sensors_results['mining']['regression_model'] = sms.OLS(num_sensors_model_df['mining_mean'], num_sensors_results['mining']['X_values']).fit()
print(num_sensors_results['mining']['regression_model'].summary())

In [ ]:
# *** change away from using generic variable names ... or define a class? 
gradient = num_sensors_results['mining']['regression_model'].params[1]
intercept = num_sensors_results['mining']['regression_model'].params[0]
rsquared = num_sensors_results['mining']['regression_model'].rsquared
pval = num_sensors_results['mining']['regression_model'].f_pvalue

In [ ]:
num_sensors_results['mining']['x_lobf'] = [num_sensors_model_df['num_sensors'].min(), num_sensors_model_df['num_sensors'].max()]
num_sensors_results['mining']['y_lobf'] = [num_sensors_results['mining']['x_lobf'][0]*gradient + intercept, num_sensors_results['mining']['x_lobf'][1]*gradient + intercept]

In [ ]:
# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))

# plt.errorbar(num_sensors_model_df['num_sensors'], num_sensors_model_df['mining_mean'], yerr=num_sensors_model_df['mining_std'], fmt='o', label="Mean mining time in blocks +/- 1σ ")
plt.scatter(num_sensors_model_df['num_sensors'], num_sensors_model_df['mining_mean'])
plt.plot(num_sensors_results['mining']['x_lobf'], num_sensors_results['mining']['y_lobf'], 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.46, 0.96))
plt.text(50, 80, "y = " + str(round(gradient, 5)) + 'x + ' + str(round(intercept, 5)))
plt.text(50,75,"p < 0.05 ")
plt.text(50,70,r"$R^2$ = " + str(round(rsquared, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Network size (number of sensors)')
_ = plt.ylabel('Number of blocks for \ntransaction to be mined')
_ = plt.title("Network size (number of sensors) against \ntransaction mean mining time (in blocks)")

# plt.savefig('./outputs/figures/num-sensors/mining-mean/scatter-OLS.svg')

### Investigating transformations

In [ ]:
num_sensors_model_df_small_networks = num_sensors_model_df[num_sensors_model_df['num_sensors'] < 200]
num_sensors_model_df_large_networks = num_sensors_model_df[num_sensors_model_df['num_sensors'] >= 200]

In [ ]:
num_sensors_model_df_large_networks['num_sensors_log'] = np.log(num_sensors_model_df_large_networks['num_sensors'])

In [ ]:
num_sensors_results['mining_log']= {}
num_sensors_results['mining_log']['pearson'] = num_sensors_model_df_large_networks[['num_sensors_log', 'mining_mean']].corr()

num_sensors_results['mining_log']['X_values'] = sms.add_constant(num_sensors_model_df_large_networks['num_sensors_log'])
num_sensors_results['mining_log']['regression_model'] = sms.OLS(num_sensors_model_df_large_networks['mining_mean'], num_sensors_results['mining_log']['X_values']).fit()
print(num_sensors_results['mining_log']['regression_model'].summary())

In [ ]:
# *** change away from using generic variable names ... or define a class? 
gradient = num_sensors_results['mining_log']['regression_model'].params[1]
intercept = num_sensors_results['mining_log']['regression_model'].params[0]
rsquared = num_sensors_results['mining_log']['regression_model'].rsquared
pval = num_sensors_results['mining_log']['regression_model'].f_pvalue

In [ ]:
num_sensors_results['mining_log']['x_lobf'] = [num_sensors_model_df_large_networks['num_sensors_log'].min(), num_sensors_model_df_large_networks['num_sensors_log'].max()]
num_sensors_results['mining_log']['y_lobf'] = [num_sensors_results['mining_log']['x_lobf'][0]*gradient + intercept, num_sensors_results['mining_log']['x_lobf'][1]*gradient + intercept]

In [ ]:
# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))

# plt.errorbar(num_sensors_model_df['num_sensors'], num_sensors_model_df['mining_mean'], yerr=num_sensors_model_df['mining_std'], fmt='o', label="Mean mining time in blocks +/- 1σ ")
plt.scatter(num_sensors_model_df_large_networks['num_sensors_log'], num_sensors_model_df_large_networks['mining_mean'])
plt.plot(num_sensors_results['mining_log']['x_lobf'], num_sensors_results['mining_log']['y_lobf'], 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.46, 0.96))
plt.text(5.3, 80, "y = " + str(round(gradient, 5)) + '* np.log(x) + ' + str(round(intercept, 5)))
plt.text(5.3,75,"p < 0.05 ")
plt.text(5.3,70,r"$R^2$ = " + str(round(rsquared, 5)))

# add R...squared, MSE, p-value
_ = plt.xlabel('Log of network size (number of sensors)')
_ = plt.ylabel('Number  of blocks for \ntransaction to be mined')
_ = plt.title("Logged network size (number of sensors) against \ntransaction mean mining time (in blocks)")

# plt.savefig('./outputs/figures/num-sensors/mining-mean/log-num-sensors-scatter-OLS.svg')

In [ ]:
num_sensors_model_df_large_networks['mining_predicted'] = num_sensors_model_df_large_networks.apply(lambda row: row['num_sensors_log'] * gradient + intercept, axis=1)
num_sensors_model_df_large_networks['mining_error'] = num_sensors_model_df_large_networks['mining_mean'] - num_sensors_model_df_large_networks['mining_predicted']


In [ ]:
plt.figure(figsize=(10,3))

plt.scatter(num_sensors_model_df_large_networks['mining_predicted'], num_sensors_model_df_large_networks['mining_error'])
_ = plt.xlabel('Predicted mean transaction time (number of blocks)')
_ = plt.ylabel('Residual error')
_ = plt.title("Residuals vs fits: The effects of network size on transaction mean mining time")

# plt.savefig('./outputs/figures/num-sensors/mining-mean/residual-errors.svg')

In [ ]:
num_sensors_model_df['mining_predicted'] = num_sensors_model_df.apply(lambda row: row['num_sensors'] * gradient + intercept, axis=1)
num_sensors_model_df['mining_error'] = num_sensors_model_df['mining_mean'] - num_sensors_model_df['mining_predicted']


In [ ]:
plt.figure(figsize=(10,3))

plt.scatter(num_sensors_model_df['mining_predicted'], num_sensors_model_df['mining_error'])
_ = plt.xlabel('Predicted mean transaction time (number of blocks)')
_ = plt.ylabel('Residual error')
_ = plt.title("Residuals vs fits: The effects of network size on transaction mean mining time")

plt.savefig('./outputs/figures/num-sensors/mining-mean/residual-errors.svg')

In [ ]:
# Deprecated
# plt.figure(figsize=(10,7))
# plt.plot(num_sensors_model_df['num_sensors'], num_sensors_model_df['mining_mean'], "bo", label="Mean number of blocks")
# plt.plot(x_lobf, y_lobf, 'r--')
# # plt.plot(np.log(num_sensors_model_df['num_sensors']), num_sensors_model_df['mining_mean-1std'], 'b.', label="+1 σ")
# # plt.plot(np.log(num_sensors_model_df['num_sensors']), num_sensors_model_df['mining_mean+1std'], 'b.', label="-1 σ")
# plt.legend(bbox_to_anchor=(1.05, 1))
# _ = plt.xlabel('Network size (number of sensors)')
# _ = plt.ylabel('Number of blocks for \ntransaction to be mined')
# plt.title("Network size by sensor count against \ntransaction mean mining time")

### Interpretation

A positive linear relationship (Pearson correlation coefficient: 0.98801) is observed between the number of sensors connecting to a blockchain and the mean mining time, measured in the number of blocks elapsed between when a transaction is submitted and when it is validated. 

Due to the limited sample size (n=7), along with constraints in the range of number of sensors swept, however, I hesitate to conclude firmly that this relationship is necessarily linear, and not a power or logarithmic relationship; further investigation is necessary. Regardless, it seems clear that increasing numbers of nodes connecting and submitting transactions to a blockchain network lengthens transaction validation times if all other factors are held constant, as hypothesized. This reaffirms the questions raised regarding the scalability of public blockchain networks for large, high transaction volume networks such as the Internet of Things. 

## Effects on informational currency

In [ ]:
plt.scatter(num_sensors_model_df['num_sensors'], num_sensors_model_df['informational_currency_mean'])

In [ ]:
small_network_info_currency_summary = pd.DataFrame(num_sensors_model_df[num_sensors_model_df['num_sensors'] == 50]['agent_vars_by_tick'][0]['informational_currency'].unstack()[30:].mean(axis=1)).rename(columns={0:'50_0'})
small_network_info_currency_summary['50_1'] = num_sensors_model_df[num_sensors_model_df['num_sensors'] == 50]['agent_vars_by_tick'][1]['informational_currency'].unstack()[30:].mean(axis=1)
small_network_info_currency_summary['50_2'] = num_sensors_model_df[num_sensors_model_df['num_sensors'] == 50]['agent_vars_by_tick'][2]['informational_currency'].unstack()[30:].mean(axis=1)

In [ ]:
for col in small_network_info_currency_summary.columns:
    pprint(calculate_adf_results(small_network_info_currency_summary[col]))

In [ ]:
calculate_adf_results(small_network_info_currency_summary['50_0'])

In [ ]:
plt.figure(figsize=(12,7))
for i, run in num_sensors_model_df[['num_sensors', 'agent_vars_by_tick']].iterrows():

    line_color = str(color_interpolator(run['num_sensors'] / num_sensors_model_df['num_sensors'].max()))
    plt.plot(run[1]['informational_currency'].unstack().mean(axis=1), label = str(run['num_sensors']), color = line_color)
    
plt.legend(bbox_to_anchor=(1.16, 1))
plt.xlabel('Block number')
plt.ylabel('Network informational currency')
plt.title('Informational currency by block number across network sizes')

# plt.savefig('./outputs/figures/num-sensors/info-currency/time-series-across-network-sizes.svg')


### Interpretation

#### Warm-up period

The expected warm-up period was observed clearly: after approximately 25-30 ticks (blocks were mined), a consistent level of informational currency was maintained, within a range of variability, for networks with fewer than 100 sensors. 

*What explains the variability observed? Is that just stochasticity? Or because record_freq = 0.3, meaning sometimes edge nodes did not transmit, as they had not recorded any new data? Because transmit_freq = 1, they did attempt to transmit every tick.*


#### Large network behavior

The observed collapse of informational currency in the larger networks (x = {320, 640}) is likely due to the fixed window of 30 blocks used to calculate informational currency. Values of 0 indicate that no data collected by any sensor within the 30 most recent blocks had been reflected on the blockchain through the transaction submission and mining process. This is in line with the observed mean mining time of > 30 for both the 320- and 640-sensor networks.

This behavior means the 280+ sensor networks will be excluded from the statistical analyses intended to identify the relationship between these two dimensions in the dataset. 

### Temporal trends

Upon reaching a threshold network size - between 60 and 100 sensors - fail to achieve what seem to be complete informational currency, in which levels are establish near 1.0. As informational currency at each tick is the ratio of data represented on-chain to the data captured in the 30 prior ticks, lower levels of informational currency were predicted to be observed in larger networks. In these iterations, greater transaction volumes, given a fixed block size (in the model a function of the block gas limit [cite line in ipynb]), mean that an individual sensor's transaction is likely to take longer to be mined into a block, especially as the size of data packets collected were homogenous. Resultingly, the lower network informational currency measures observed are in line with the hypothesis.

However, the apparent non-stationarity of informational currency measures of these larger networks over time was not expected. As early transactions are broadcasted to the miners and added to the mempool, they are mined - within the first few blocks informational currency is very high because few ticks have passed, meaning little data has been collected at the edge, and transactions are being validated each tick. However, this increase in informational currency begins to slow as the network reaches a point where """???""" ...

There is no obvious reason for the informational currency of the network to initially experience a higher proportion of data captured being validated and written to the blockchain, but for that proportion to decrease as more transactions are submitted. This pattern, of overshooting, then decreasing over time, may be due to quirks in model design and calibration, or to some element of system functioning that is unsustainable once some threshold of usage is exceeded. (Footnote: Here we raise a critique of agent-based modeling as an investigatory approach: it is impossible to tell whether emergent complexity - precisely what the technique is meant to simulate - is a result of intended agent interactions or of just some model design decision without validating the model data collected from observed reality. It is therefore difficult to make any firm assertions about complex system behavior based on analysis of model results. ***Justify lack of calibration dataset despite its availability?***) If the negative trend of informational currency in larger networks is a genuine result, perhaps it can be explained by the fact that the number of unvalidated transaction is growing at a rate faster than it is being reduced, so over time a smaller percentage of the prior 30 block's submissions are validated. However, this still does not explain why it achieves are 
perhaps it is due to the growing disproportion between the number of transactions submitted

As an additional curiosity, one network size - 150 sensors - did not exhibit the negative correlation between time and informational currency, despite the model run simulating the next smaller network (100 sensors) and one simulating the next larger (210 sensors) both experiencing the decreasing informational currency values as time increased. 

Clearly much investigation is warranted to understand the effects of network loads on blockchain behavior over time. 

This negative relationship is difficult to interpret. 

The unexpected result is the non-stationarity observed in the plots of informational currency measures over time. The negative relationship - in some cases appearing linear, in others, perhaps asymptotic, 

In [ ]:
# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(num_sensors_model_df['num_sensors'], num_sensors_model_df['informational_currency_post_warmup_mean'])#, yerr=num_sensors_model_df['informational_currency_post_warmup_std'], fmt='o', label="Mean informational currency +/- 1σ ")
plt.legend(bbox_to_anchor=(0.96, 0.96))
# plt.text(20, 130, "y = " + str(round(gradient, 5)) + 'x + ' + str(round(intercept, 5)))
# plt.text(20,120,"p = " + str(round(pval, 5)))
# plt.text(20,110,r"$R^2$ = " + str(round(rsquared, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Network size (number of sensors)')
_ = plt.ylabel('Informational currency')
_ = plt.title("Network size (number of sensors) against \nmean informational currency")

#### Interpretation

A convex negative nonlinear relationship appears to exist between network size and the measure of informational currency employed here. While a negative relationship was hypothesized, the nonlinear nature was unexpected. Further modeling and statistical analysis is warranted to better understand the dynamics of how network size affects how well the state of the cloud ledger reflects the state at the edge. 

Why?

### Assessing stationarity: informational currency over time

In order to analyze temporal autocorrelation and seasonality in measures of informational currency over time (after the warm-up period), 

ACF plot
PACF plots
Augmented Dickey-Fuller test

In [ ]:
for i in range(len(num_sensors_model_df['agent_vars_by_tick'])):
    t = "Autocorrelation of informational currency, \n" + str(num_sensors_model_df.loc[i, 'num_sensors']) + " sensors"
    _ = plot_acf(num_sensors_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values, title=t)

In [ ]:
for i in range(len(num_sensors_model_df['agent_vars_by_tick'])):
    t = "Partial autocorrelation of informational currency, \n" + str(num_sensors_model_df.loc[i, 'num_sensors']) + " sensors"

    _ = plot_pacf(num_sensors_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values, title=t)

In [ ]:
calculate_adf_results(num_sensors_model_df['agent_vars_by_tick'][1]['informational_currency'].unstack().mean(axis=1).values).keys()

In [ ]:
num_sensors_adf_df = pd.DataFrame(columns=['num_sensors', 'adf_statistic', 'p_value', '1%_critical_value', '5%_critical_value', '10%_critical_value'])

for i in range(len(num_sensors_model_df['agent_vars_by_tick'])):
    adf_results = calculate_adf_results(num_sensors_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values)
    adf_results['num_sensors'] = num_sensors_model_df.loc[i, 'num_sensors']

    num_sensors_adf_df = num_sensors_adf_df.append(pd.DataFrame([adf_results], columns=adf_results.keys()))

num_sensors_adf_df = num_sensors_adf_df.reset_index().drop(columns=['index'])
num_sensors_adf_df


In [ ]:
# Adapted from CEGExxx: Spatial Temporal Data Analysis and Data Mining final project


# *** convert to dataframe ... 
for i in range(len(num_sensors_model_df['agent_vars_by_tick'])):

    adf_num_sensors = adfuller(num_sensors_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values)
    print("Augmented Dickey-Fuller test, " + str(num_sensors_model_df.loc[i, 'num_sensors']) + " sensors")
    print('ADF Statistic: %f' % adf_num_sensors[0])
    print('p-value: %f' % adf_num_sensors[1])
    print('Critical Values:')
    for key, value in adf_num_sensors[4].items():
        print('\t%s: %.3f' % (key, value))
    print()

### Interpretation

The p-value of <0.05 on each of the Augmented Dickey-Fuller tests performed on the time series data representing informational currency serves as statistically significant evidence of what was visually observed in both the series plot and the autocorrelation function plot: the series do not have unit roots, and are stationary processes. This is important because it suggests that, with all other factors remaining constant, for a given transaction volume per block informational currency establishes a stable level. The variation was observed around that mean was based on stochasticity of transaction transmission: if a sensor did not transmit for an unusually long period, its informational currency level likely increased as transactions already submitted to the chain were validated, increasing the proportion reflected on chain of data recorded at the edge. The important observation is the stationarity of the metric: if blockchain transaction volume processing capacity adapts in response to submitted volumes, or submitted volumes are adjusted through incentivization adjustments (i.e. gas costs), sustainable operation of the blockchain appears possible, in this regard.

## Effects on gwei spent

Perhaps the primary constraint unique to public blockchains is the requirement that users pay for compute and storage space used. This is a necessary attribute of the system, designed to prevent intentional abuse or unintentional misuse, which seems to be inevitable in permissionless, anonymous systems. 



In [ ]:
plt.figure(figsize=(10,7))

plt.scatter(num_sensors_model_df['num_sensors'], num_sensors_model_df['gwei_spent_mean'])

_ = plt.xlabel('Network size (number of sensors)')
_ = plt.ylabel('Mean gwei spent per sensor per block')
_ = plt.title("Network size (number of sensors) against \n mean gwei spent per sensor per block")

# plt.savefig('./outputs/figures/num-sensors/gwei-spent/ols-scatter-xtransformed.svg')

In [ ]:
num_sensors_model_df[['num_sensors'] + [col for col in num_sensors_model_df.columns if 'gwei' in col] ].groupby('num_sensors').std()[[ 'gwei_spent_mean']]

In [ ]:
plt.scatter(num_sensors_model_df['num_sensors'], num_sensors_model_df['gwei_spent_mean'], label='Model run') #, yerr=num_sensors_model_df['gwei_spent_std'], fmt='o', label="Mean gwei spent per sensor +/- 1σ ")


In [ ]:
num_sensors_model_df['gwei_spent_mean'].describe()

In [ ]:
num_sensors_model_df['gwei_spent_mean'].hist()

In [ ]:
X_values_gwei_untransformed = sms.add_constant(num_sensors_model_df['num_sensors'])
regression_model_gwei_spent_untransformed = sms.OLS(num_sensors_model_df['gwei_spent_mean'], X_values_gwei_untransformed).fit()
print(regression_model_gwei_spent_untransformed.summary())

In [ ]:
num_sensors_model_df['num_sensors_logged'] = np.log(num_sensors_model_df['num_sensors'])
num_sensors_model_df['gwei_spent_mean_logged'] = np.log(num_sensors_model_df['gwei_spent_mean'])

In [ ]:
pearson_gwei_xtransformed = num_sensors_model_df[['num_sensors_logged', 'gwei_spent_mean']].corr()

X_values_gwei_xtransformed = sms.add_constant(num_sensors_model_df['num_sensors_logged'])
regression_model_gwei_spent_xtransformed = sms.OLS(num_sensors_model_df['gwei_spent_mean'], X_values_gwei_xtransformed).fit()
print(regression_model_gwei_spent_xtransformed.summary())

In [ ]:
gradient_gwei = regression_model_gwei_spent_xtransformed.params[1]
intercept_gwei = regression_model_gwei_spent_xtransformed.params[0]
rsquared_gwei = regression_model_gwei_spent_xtransformed.rsquared
pval_gwei = regression_model_gwei_spent_xtransformed.f_pvalue

In [ ]:
x_lobf_gwei_xtransformed = [num_sensors_model_df['num_sensors_logged'].min(), num_sensors_model_df['num_sensors_logged'].max()]
y_lobf_gwei_xtransformed = [x_lobf_gwei_xtransformed[0] * gradient_gwei + intercept_gwei, x_lobf_gwei_xtransformed[1] * gradient_gwei + intercept_gwei]

In [ ]:
y_lobf_gwei_xtransformed

In [ ]:

# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(num_sensors_model_df['num_sensors_logged'], num_sensors_model_df['gwei_spent_mean'], label='Model run') #, yerr=num_sensors_model_df['gwei_spent_std'], fmt='o', label="Mean gwei spent per sensor +/- 1σ ")

# plt.errorbar(num_sensors_model_df['num_sensors'], num_sensors_model_df['gwei_spent_mean'], yerr=num_sensors_model_df['gwei_spent_std'], fmt='o', label="Mean gwei spent per sensor +/- 1σ ")
plt.plot(x_lobf_gwei_xtransformed, y_lobf_gwei_xtransformed, 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.96, 0.96))
plt.text(5.2, 11.5, "y = " + str(round(gradient_gwei, 5)) + 'x + ' + str(round(intercept_gwei, 5)))
plt.text(5.2,11.4, "p < 0.05")
plt.text(5.2,11.3, r"$R^2$ = " + str(round(rsquared_gwei, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Natural log of network size (number of sensors)')
_ = plt.ylabel('Mean gwei spent per sensor per block')
_ = plt.title("Log of network size (number of sensors) against \n mean gwei spent per sensor per block")

# plt.savefig('./outputs/figures/num-sensors/gwei-spent/ols-scatter-xtransformed.svg')

In [ ]:
num_sensors_model_df

In [ ]:
num_sensors_model_df['gwei_spent_predicted'] = num_sensors_model_df.apply(lambda row: row['num_sensors_logged'] * gradient_gwei + intercept_gwei, axis=1)
num_sensors_model_df['gwei_spent_error'] = num_sensors_model_df['gwei_spent_mean'] - num_sensors_model_df['gwei_spent_predicted']

In [ ]:
plt.figure(figsize=(10,3))

plt.scatter(num_sensors_model_df['gwei_spent_predicted'], num_sensors_model_df['gwei_spent_error'])
_ = plt.xlabel('Predicted expenditures (gwei per sensor per block)')
_ = plt.ylabel('Residual error')
_ = plt.title("Residuals vs fits: The effects of log of network size on gwei spent per sensor per block")

# plt.savefig('./outputs/figures/num-sensors/gwei-spent/residual-errors.svg')

#### Interpretation 

An negative relationship is observed: as network size increases, average gwei spent per sensor per block decreases. This was expected given that each block has a fixed gas limit, here placing a limit on the amount of data that can be written to the blockchain each block validation - as the number of sensors attempting to submit data to be stored on chain increases, the likelihood of each sensor's transactions being validated, and contained data being written, decreases. Costs in gwei incurred are only deducted from the sensor's externally owned account upon transaction validation - the decrease in gwei spent relates closely to the decrease in informational currency observed. (Common cause)

While the p-value calculated for the line of best fit using OLS regression is below the 0.05 threshold set to accept significance, the non-normal distribution of residual errors suggests that a non-linear model may fit the data better. Further research into the nature of the relationship between network size (in effect, transaction volumes) and cost to each network participant is warranted. This is a point where deploying heterogeneous agents submitting transactions at differing frequencies and writing data of varying volumes could provide even further insight into network scalability. 

In [ ]:
num_sensors_model_df[['num_sensors', 'gwei_spent_mean']]

## The effects of record volumes on dependent variables

In [1]:
record_bytes_model_df = get_model_vars_df_from_pickle('./data/final/batch_run_1_record_bytes.pkl')

NameError: name 'get_model_vars_df_from_pickle' is not defined

### Effects on mining dynamics

In [ ]:
plt.scatter(record_bytes_model_df['record_bytes'], record_bytes_model_df['mining_mean'])

In [ ]:
record_bytes_model_df[ 'mining_mean'].hist()

In [ ]:
record_bytes_model_summary = record_bytes_model_df[['record_bytes', 'mining_mean']].describe()

In [ ]:
record_bytes_model_summary['mining_mean']

In [ ]:
record_bytes_tukey_fence = record_bytes_model_summary.loc['mean', 'mining_mean'] + (1.5 * record_bytes_model_summary.loc['75%', 'mining_mean'] - record_bytes_model_summary.loc['25%', 'mining_mean'])

In [ ]:
record_bytes_model_df_mining = record_bytes_model_df[record_bytes_model_df['mining_mean'] < record_bytes_tukey_fence]

In [ ]:
record_bytes_model_df_mining[['record_bytes', 'mining_mean']].corr()

In [ ]:
pearson_record_bytes = record_bytes_model_df_mining[['record_bytes', 'mining_mean']].corr()

In [ ]:
record_bytes_model_df_mining[['record_bytes', 'mining_mean']].describe()

In [ ]:
plt.scatter(record_bytes_model_df_mining['record_bytes'], record_bytes_model_df_mining['mining_mean'])

In [ ]:
X_values_record_bytes = sms.add_constant(record_bytes_model_df_mining['record_bytes'])
regression_model_record_bytes = sms.OLS(record_bytes_model_df_mining['mining_mean'], X_values_record_bytes).fit()
print(regression_model_record_bytes.summary())

In [ ]:
gradient_record_bytes = regression_model_record_bytes.params[1]
intercept_record_bytes = regression_model_record_bytes.params[0]
rsquared_record_bytes = regression_model_record_bytes.rsquared
pval_record_bytes = regression_model_record_bytes.f_pvalue

In [ ]:

x_lobf_record_bytes = [record_bytes_model_df_mining['record_bytes'].min(), record_bytes_model_df_mining['record_bytes'].max()]
y_lobf_record_bytes = [x_lobf_record_bytes[0]*gradient_record_bytes + intercept_record_bytes, x_lobf_record_bytes[1]*gradient_record_bytes + intercept_record_bytes]


In [ ]:

# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
# plt.errorbar(record_bytes_model_df['record_bytes'], record_bytes_model_df['mining_mean'], yerr=record_bytes_model_df['mining_std'], fmt='o', label="Mean mining time in blocks +/- 1σ ")
plt.scatter(record_bytes_model_df_mining['record_bytes'], record_bytes_model_df_mining['mining_mean'], label='Model run')
plt.plot(x_lobf_record_bytes, y_lobf_record_bytes, 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.44, 0.96))
plt.text(-5, 14, "y = " + str(round(gradient_record_bytes, 5)) + 'x + ' + str(round(intercept_record_bytes, 5)))
plt.text(-5,13, "p < 0.05")
plt.text(-5, 12, r"$R^2$ = " + str(round(rsquared_record_bytes, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Number of bytes recorded upon empirical observation')
_ = plt.ylabel('Number of blocks for \ntransaction to be mined')
_ = plt.title("Sensor data capture volumes (bytes per recording) against \ntransaction mean mining time (blocks)")

# plt.savefig('./outputs/figures/record-vol/mining-mean/scatter-ols.svg')

In [ ]:
record_bytes_model_df_mining['mining_predicted'] = record_bytes_model_df_mining.apply(lambda row: row['record_bytes'] * gradient_record_bytes + intercept_record_bytes, axis=1)
record_bytes_model_df_mining['mining_error'] = record_bytes_model_df_mining['mining_mean'] - record_bytes_model_df_mining['mining_predicted']

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter( record_bytes_model_df_mining['mining_predicted'], record_bytes_model_df_mining['mining_error'])
plt.xlabel('Predicted mean mining time')
plt.ylabel('Residual error')
plt.title('Mean mining time residual errors versus \nvalues predicted from OLS simple linear regression')

# plt.savefig('./outputs/figures/record-vol/mining-mean/residuals-fits.svg')

In [ ]:
plt.figure(figsize=(10,4))
record_bytes_model_df_mining['mining_error'].hist()
plt.title('Distribution of residual errors of mean mining times')
plt.xlabel('Residual error')
plt.ylabel('Number of model runs')

# plt.savefig('./outputs/figures/record-vol/mining-mean/residuals-histogram.svg')

In [ ]:
record_bytes_model_df_mining[['mining_error','mining_predicted']].corr()

### Effects of data capture volumes on network informational currency

In [ ]:
plt.figure(figsize=(12,7))
for i, run in record_bytes_model_df[['agent_vars_by_tick']].iterrows():
    plt.plot(run[0]['informational_currency'].unstack().mean(axis=1),\
             label=record_bytes_model_df.loc[i, 'record_bytes'], color=str(color_interpolator(record_bytes_model_df.loc[i, 'record_bytes'] / record_bytes_model_df['record_bytes'].max()))) #, cmap='gray', c=record_bytes_model_df['record_bytes'] )

plt.legend(bbox_to_anchor=(1.15, 1))
plt.title('Informational currency over time across a range of data capture volumes')
plt.xlabel('Block number')
_ = plt.ylabel("Network informational currency")

# plt.savefig('./outputs/figures/record-vol/info-currency/time-series-across-record-volumes.svg')

In [ ]:
# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(record_bytes_model_df['record_bytes'], record_bytes_model_df['informational_currency_post_warmup_mean'])#, yerr=record_bytes_model_df['informational_currency_post_warmup_std'], fmt='o', label="Mean informational currency +/- 1σ ")

# plt.errorbar(record_bytes_model_df['record_bytes'], record_bytes_model_df['informational_currency_post_warmup_mean'], yerr=record_bytes_model_df['informational_currency_post_warmup_std'], fmt='o', label="Mean informational currency +/- 1σ ")
plt.legend(bbox_to_anchor=(0.96, 0.96))
# plt.text(20, 130, "y = " + str(round(gradient, 5)) + 'x + ' + str(round(intercept, 5)))
# plt.text(20,120,"p = " + str(round(pval, 5)))
# plt.text(20,110,r"$R^2$ = " + str(round(rsquared, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Data capture volume (bytes)')
_ = plt.ylabel('Informational currency')
_ = plt.title("Data capture volumes (bytes) against \nmean calculated informational currency")

# plt.savefig('./outputs/figures/record-vol/info-currency/scatter.svg')

In [ ]:
record_bytes_model_df[record_bytes_model_df['record_bytes'] > 300][['record_bytes'] + [col for col in record_bytes_model_df if 'info' in col and 'mean'  in col]]

In [ ]:
for i in range(len(record_bytes_model_df['agent_vars_by_tick'])):
    t = "Autocorrelation of informational currency, \n" + str(record_bytes_model_df.loc[i, 'record_bytes']) + " bytes recorded"
    _ = plot_acf(record_bytes_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values, title=t)

Some cyclicity may appear in some of the higher recording volumes. 

This may be explained by the reduction in the number of transactions that can be validated per block. As transactions wait in the mempool, informational currency will decrease on each block - until it is validated, which would prompt a leap. However, informational currency is a measure of the state of the network, not an individual edge node (sensor) ...???

In [ ]:
for i in range(len(record_bytes_model_df['agent_vars_by_tick'])):
    t = "Partial autocorrelation of informational currency, \n" + str(record_bytes_model_df.loc[i, 'record_bytes']) + " bytes recorded each observation"

    _ = plot_pacf(record_bytes_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values, title=t)

In [ ]:
# Adapted from CEGExxx: Spatial Temporal Data Analysis and Data Mining final project

for i in range(len(record_bytes_model_df['agent_vars_by_tick'])):

    adf_num_sensors = adfuller(record_bytes_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values)
    print("Augmented Dickey-Fuller test, " + str(record_bytes_model_df.loc[i, 'record_bytes']) + " bytes recorded per observation")
    print('ADF Statistic: %f' % adf_num_sensors[0])
    print('p-value: %f' % adf_num_sensors[1])
    print('Critical Values:')
    for key, value in adf_num_sensors[4].items():
        print('\t%s: %.3f' % (key, value))
    print()

Only a few p-values (241 bytes per obs, 181 bytes per obs) on the Augmented Dickey-Fuller tests indicated the rejection of the null hypothesis and the potential of the series having a unit root, indicating nonstationarity. 

## Gwei spent

In [ ]:
plt.scatter(record_bytes_model_df['record_bytes'], record_bytes_model_df['gwei_spent_post_warmup_mean'])

In [ ]:
record_bytes_seg_0 = record_bytes_model_df[(record_bytes_model_df['record_bytes'] < 240) & (record_bytes_model_df['record_bytes'] > 50) ]
record_bytes_seg_1 = record_bytes_model_df[(record_bytes_model_df['record_bytes'] > 240) & (record_bytes_model_df['record_bytes'] < 322)]
record_bytes_seg_2 = record_bytes_model_df[record_bytes_model_df['record_bytes'] > 321]

In [ ]:
plt.errorbar(record_bytes_seg_0['record_bytes'], record_bytes_seg_0['gwei_spent_mean'], yerr=record_bytes_seg_0['gwei_spent_std'])

In [ ]:
record_bytes_seg_0[['record_bytes','gwei_spent_mean']].corr()

In [ ]:
pearson_gwei_record_bytes = record_bytes_seg_1[['record_bytes', 'gwei_spent_mean']].corr()

X_values_gwei_record_bytes = sms.add_constant(record_bytes_seg_1['record_bytes'])
regression_model_gwei_spent_record_bytes = sms.OLS(record_bytes_seg_1['gwei_spent_mean'], X_values_gwei_record_bytes).fit()
print(regression_model_gwei_spent_record_bytes.summary())

In [ ]:
gradient_gwei_record_bytes = regression_model_gwei_spent_record_bytes.params[1]
intercept_gwei_record_bytes = regression_model_gwei_spent_record_bytes.params[0]
rsquared_gwei_record_bytes = regression_model_gwei_spent_record_bytes.rsquared
pval_gwei_record_bytes = regression_model_gwei_spent_record_bytes.f_pvalue

In [ ]:
x_lobf_gwei_record_bytes = [record_bytes_seg_1['record_bytes'].min(), record_bytes_seg_1['record_bytes'].max()]
y_lobf_gwei_record_bytes = [x_lobf_gwei_record_bytes[0] * gradient_gwei_record_bytes + intercept_gwei_record_bytes, x_lobf_gwei_record_bytes[1] * gradient_gwei_record_bytes + intercept_gwei_record_bytes]

In [ ]:

# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(record_bytes_seg_1['record_bytes'], record_bytes_seg_1['gwei_spent_mean'])

plt.plot(x_lobf_gwei_record_bytes, y_lobf_gwei_record_bytes, 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.96, 0.96))
# plt.text(400, 200000, "y = " + str(round(gradient_gwei_record_bytes, 5)) + 'x + ' + str(round(intercept_gwei_record_bytes, 5)))
# plt.text(400,180000,"p = " + str(round(pval_gwei_record_bytes, 5)))
# plt.text(400,160000,r"$R^2$ = " + str(round(rsquared_gwei_record_bytes, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Data capture volumes (bytes per observation)')
_ = plt.ylabel('Mean gwei spent per sensor per block')
_ = plt.title("Data capture volumes (bytes per observation, segmented) versus \nmean gwei spent per sensor per block")

# plt.savefig('./outputs/figures/record-vol/gwei-spent/scatter-ols-segment-241-321.svg')

In [ ]:
pearson_gwei_record_bytes.loc['record_bytes', 'gwei_spent_mean']

In [ ]:
pearson_gwei_record_bytes = record_bytes_seg_2[['record_bytes', 'gwei_spent_mean']].corr()

X_values_gwei_record_bytes = sms.add_constant(record_bytes_seg_2['record_bytes'])
regression_model_gwei_spent_record_bytes = sms.OLS(record_bytes_seg_2['gwei_spent_mean'], X_values_gwei_record_bytes).fit()
print(regression_model_gwei_spent_record_bytes.summary())

In [ ]:
gradient_gwei_record_bytes = regression_model_gwei_spent_record_bytes.params[1]
intercept_gwei_record_bytes = regression_model_gwei_spent_record_bytes.params[0]
rsquared_gwei_record_bytes = regression_model_gwei_spent_record_bytes.rsquared
pval_gwei_record_bytes = regression_model_gwei_spent_record_bytes.f_pvalue

In [ ]:
x_lobf_gwei_record_bytes = [record_bytes_seg_2['record_bytes'].min(), record_bytes_seg_2['record_bytes'].max()]
y_lobf_gwei_record_bytes = [x_lobf_gwei_record_bytes[0] * gradient_gwei_record_bytes + intercept_gwei_record_bytes, x_lobf_gwei_record_bytes[1] * gradient_gwei_record_bytes + intercept_gwei_record_bytes]

In [ ]:

# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(record_bytes_seg_2['record_bytes'], record_bytes_seg_2['gwei_spent_mean'])

plt.plot(x_lobf_gwei_record_bytes, y_lobf_gwei_record_bytes, 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.96, 0.96))
# plt.text(400, 200000, "y = " + str(round(gradient_gwei_record_bytes, 5)) + 'x + ' + str(round(intercept_gwei_record_bytes, 5)))
# plt.text(400,180000,"p = " + str(round(pval_gwei_record_bytes, 5)))
# plt.text(400,160000,r"$R^2$ = " + str(round(rsquared_gwei_record_bytes, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Data capture volumes (bytes per observation)')
_ = plt.ylabel('Mean gwei spent per sensor per block')
_ = plt.title("Data capture volumes (bytes per observation, segmented) versus \nmean gwei spent per sensor per block")

# plt.savefig('./outputs/figures/record-vol/gwei-spent/scatter-ols-segment-341-501.svg')

In [ ]:
plt.scatter(record_bytes_seg_1['record_bytes'], record_bytes_seg_1['gwei_spent_post_warmup_mean'])

In [ ]:
pearson_gwei_record_bytes = record_bytes_model_df[['record_bytes', 'gwei_spent_mean']].corr()
pearson_gwei_record_bytes

In [ ]:
pearson_gwei_record_bytes = record_bytes_model_df[['record_bytes', 'gwei_spent_mean']].corr()

X_values_gwei_record_bytes = sms.add_constant(record_bytes_model_df['record_bytes'])
regression_model_gwei_spent_record_bytes = sms.OLS(record_bytes_model_df['gwei_spent_mean'], X_values_gwei_record_bytes).fit()
print(regression_model_gwei_spent_record_bytes.summary())

In [ ]:
gradient_gwei_record_bytes = regression_model_gwei_spent_record_bytes.params[1]
intercept_gwei_record_bytes = regression_model_gwei_spent_record_bytes.params[0]
rsquared_gwei_record_bytes = regression_model_gwei_spent_record_bytes.rsquared
pval_gwei_record_bytes = regression_model_gwei_spent_record_bytes.f_pvalue

In [ ]:
x_lobf_gwei_record_bytes = [record_bytes_model_df['record_bytes'].min(), record_bytes_model_df['record_bytes'].max()]
y_lobf_gwei_record_bytes = [x_lobf_gwei_record_bytes[0] * gradient_gwei_record_bytes + intercept_gwei_record_bytes, x_lobf_gwei_record_bytes[1] * gradient_gwei_record_bytes + intercept_gwei_record_bytes]

In [ ]:

# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(record_bytes_model_df['record_bytes'], record_bytes_model_df['gwei_spent_mean'])
# plt.errorbar(record_bytes_model_df['record_bytes'], record_bytes_model_df['gwei_spent_mean'], yerr=record_bytes_model_df['gwei_spent_std'], fmt='o', label="Mean gwei spent per sensor +/- 1σ ")
plt.plot(x_lobf_gwei_record_bytes, y_lobf_gwei_record_bytes, 'r--', label="Line of best fit (OLS linear regression)" )
plt.legend(bbox_to_anchor=(0.96, 0.96))
plt.text(400, 200000, "y = " + str(round(gradient_gwei_record_bytes, 5)) + 'x + ' + str(round(intercept_gwei_record_bytes, 5)))
plt.text(400,180000,"p = " + str(round(pval_gwei_record_bytes, 5)))
plt.text(400,160000,r"$R^2$ = " + str(round(rsquared_gwei_record_bytes, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Data capture volumes (bytes per observation)')
_ = plt.ylabel('Mean gwei spent per sensor per block')
_ = plt.title("Data capture volumes (bytes per observation) versus \nmean gwei spent per sensor per block")

Extremely low Pearson correlation coefficient suggests that no relationship exists between the mean gwei spent per sensor per block ...

This seems weird?

## The effects of record frequency on dependent variables

In [3]:
record_freq_model_df = get_model_vars_df_from_pickle('./data/final/batch_run_1_record_freq.pkl')

In [7]:
record_freq_model_df['model_vars_by_tick'][0]

,active_sensors
0,40
1,40
2,40
3,40
4,40
5,40
6,40
7,40
8,40
9,40


In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(record_freq_model_df['record_freq'], record_freq_model_df['mining_mean'])
plt.xlabel('Record frequency (probability per block)')
plt.ylabel('Mean mining time per transaction (blocks)')
plt.title('Record frequency vs mean mining time per transaction')
# plt.savefig('./outputs/figures/record-freq/mining-mean/scatter-untransformed.svg')

In [ ]:
record_freq_model_df['mining_mean_sqrt'] = np.sqrt(record_freq_model_df['mining_mean'])
record_freq_model_df['mining_mean_sqrt_sqrt'] = np.sqrt(record_freq_model_df['mining_mean_sqrt'])

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(record_freq_model_df['record_freq'], np.sqrt(record_freq_model_df['mining_mean_sqrt']))
plt.xlabel('Record frequency (probability per block)')
plt.ylabel('Square root of mean mining time per transaction (blocks)')
plt.title('Record frequency vs transformed mean mining time per transaction')
# plt.savefig('./outputs/figures/record-freq/mining-mean/scatter-ytransformed.svg')

In [ ]:
record_freq_model_df_subset = record_freq_model_df[record_freq_model_df['record_freq'] > 0.2]

In [ ]:
record_freq_OLS = SimpleOLSRegression(record_freq_model_df_subset, 
                                      'record_freq', 'Record frequency (probability per block)',
                                     'mining_mean_sqrt', 'Square root of mean transaction mining time (blocks)')

In [ ]:
record_freq_OLS.scatter_OLS()#'./outputs/figures/record-freq/mining-mean/scatter-ols-ysingletransform.svg')

In [ ]:
record_freq_OLS.plot_residuals_fits()#'./outputs/figures/record-freq/mining-mean/residuals-fits-ysingletransform.svg')

In [ ]:
record_freq_OLS_double = SimpleOLSRegression(record_freq_model_df_subset, 
                                      'record_freq', 'Record frequency (probability per block)',
                                     'mining_mean_sqrt_sqrt', 'Double square root of mean transaction mining time (blocks)')

In [ ]:
record_freq_OLS_double.scatter_OLS()#'./outputs/figures/record-freq/mining-mean/scatter-ols-ydoubletransform.svg')

In [ ]:
record_freq_OLS_double.plot_residuals_fits()#'./outputs/figures/record-freq/mining-mean/residuals-fits-ydoubletransform.svg')

### Effects of record frequency on informational currency

In [ ]:
plt.figure(figsize=(12,7))
for i, run in record_freq_model_df[['agent_vars_by_tick']].iterrows():
    plt.plot(run[0]['informational_currency'].unstack().mean(axis=1),\
             label=record_freq_model_df.loc[i, 'record_freq'], 
             color=str(color_interpolator(
                 (record_freq_model_df.loc[i, 'record_freq'] - record_freq_model_df['record_freq'].min()) / 
                 (record_freq_model_df['record_freq'].max() - record_freq_model_df['record_freq'].min())))) #, cmap='gray', c=record_bytes_model_df['record_bytes'] )

plt.legend(bbox_to_anchor=(1.15, 1))
plt.title('Informational currency over time across a range of data capture frequencies')
plt.xlabel('Block number')
_ = plt.ylabel("Network informational currency")

# plt.savefig('./outputs/figures/record-freq/info-currency/time-series-across-record-volumes.svg')

In [ ]:
# Visualizing observations and OLS regression
plt.figure(figsize=(10,7))
plt.scatter(record_freq_model_df['record_freq'], record_freq_model_df['informational_currency_post_warmup_mean'])#, yerr=record_bytes_model_df['informational_currency_post_warmup_std'], fmt='o', label="Mean informational currency +/- 1σ ")

plt.legend(bbox_to_anchor=(1.5, 0.96))
# plt.text(20, 130, "y = " + str(round(gradient, 5)) + 'x + ' + str(round(intercept, 5)))
# plt.text(20,120,"p = " + str(round(pval, 5)))
# plt.text(20,110,r"$R^2$ = " + str(round(rsquared, 5)))

# add Rsquared, MSE, p-value
_ = plt.xlabel('Data capture frequency (probability)')
_ = plt.ylabel('Informational currency')
_ = plt.title("Data capture frequencies against \nmean calculated informational currency")

# plt.savefig('./outputs/figures/record-freq/info-currency/mean-post-warmup-scatter.svg')

#### Assessing stationarity of time series

In [ ]:
record_freq_adf_df = pd.DataFrame(columns=['record_freq', 'adf_statistic', 'p_value', '1%_critical_value', '5%_critical_value', '10%_critical_value'])

for i in range(len(record_freq_model_df['agent_vars_by_tick'])):
    adf_results = calculate_adf_results(record_freq_model_df['agent_vars_by_tick'][i]['informational_currency'].unstack().mean(axis=1).values)
    adf_results['record_freq'] = record_freq_model_df.loc[i, 'record_freq']

    record_freq_adf_df = record_freq_adf_df.append(pd.DataFrame([adf_results], columns=adf_results.keys()))

record_freq_adf_df = record_freq_adf_df.reset_index().drop(columns=['index'])



In [ ]:
calculate_adf_results(record_freq_model_df['agent_vars_by_tick'][19]['informational_currency'].unstack().mean(axis=1).values)

In [ ]:
plt.scatter(record_freq_adf_df['record_freq'], record_freq_adf_df['p_value'])

In [ ]:
record_freq_adf_df[record_freq_adf_df['p_value'] > 0.05]

### Effects of record frequency on gwei spent

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(record_freq_model_df['record_freq'], record_freq_model_df['gwei_spent_mean'], label='Model iteration')
plt.title('Record frequencies vs mean gwei spent per sensor per block')
plt.xlabel('Record frequency (probability per block)')
_ = plt.ylabel('Mean gwei spent per sensor per block')

# plt.savefig('./outputs/figures/record-freq/gwei-spent/scatter.svg')

In [ ]:
record_freq_model_df_subset = record_freq_model_df[record_freq_model_df['record_freq'] < 0.5]

In [ ]:
record_freq_model_df_subset['gwei_spent_mean_log'] = np.log(record_freq_model_df_subset['gwei_spent_mean'])

In [ ]:
record_freq_gwei_OLS = SimpleOLSRegression(record_freq_model_df_subset, 'record_freq', "Record frequency (probability per block)", "gwei_spent_mean", "Mean gwei spent per sensor per block")

In [ ]:
record_freq_gwei_OLS.scatter_OLS()#'./outputs/figures/record-freq/gwei-spent/scatter-ols-subset.svg')

In [ ]:
record_freq_gwei_OLS.plot_residuals_fits()#'./outputs/figures/record-freq/gwei-spent/residuals-fits-subset.svg')

In [ ]:
record_freq_model_df[['record_freq', 'gwei_spent_mean']].describe()